Carregando o pandas, numpy e imprimindo a versão do pandas

In [16]:
import numpy as np
import pandas as pd
import gzip
import io
import requests

PANDA_VERSION = pd.__version__
PROPOSICAO_JSON_DS_FILE = 'https://github.com/bozoh/meu-legislador/raw/master/data/ds/proposicoes.json.gz'
DEPUTADOS_JSON_DS_FILE = 'https://github.com/bozoh/meu-legislador/raw/master/data/ds/deputados.json.gz'

In [12]:
PANDA_VERSION

'0.22.0'

Carregando os dados de todas as proposições (PL, PLP, PEC, PLV, PDC, MPV) da câmara dos deputados 
a partit de 1/1/1984


In [68]:
web_response = requests.get(PROPOSICAO_JSON_DS_FILE, timeout=30, stream=True)
f = io.BytesIO(web_response.content)

with gzip.GzipFile(fileobj=f) as fh:
    df_prep = pd.read_json(fh, orient='split')

Carregando dados dos depudados

In [67]:
web_response = requests.get(DEPUTADOS_JSON_DS_FILE, timeout=30, stream=True)
f = io.BytesIO(web_response.content)

with gzip.GzipFile(fileobj=f) as fh:
    df_dep = pd.read_json(fh, orient='split')

In [ ]:
Nesse estudo só quero o nome dos deputados

In [69]:
df_dep = df_dep[['id', 'nomeCivil']]
df_dep

,id,nomeCivil
0,65551,RICARDO WAGNER DE CARVALHO LAGO
1,139285,LÍDICE DA MATA E SOUZA
2,73761,NELSON TRAD
3,73764,ABELARDO LUIZ LUPION MELLO
4,73765,AFFONSO ALVES DE CAMARGO NETO
5,73768,DILCEU JOÃO SPERAFICO
6,73772,HERMES PARCIANELLO
7,73775,JOSÉ RODRIGUES BORBA
8,73776,JOSÉ MOHAMED JANENE
9,73777,LUCIANO PIZZATTO


Acrescentando a coluna idAutor, obtido pela coluna uriAutores, que no final tem o id do deputado

In [70]:
df_prep['idAutor'] = df_prep.uriAutores.map(lambda x: int(x.replace('https://dadosabertos.camara.leg.br/api/v2/deputados/','')) if x else np.nan)

Acrescentando o nome do autor nas proposições

In [71]:
df_prep=pd.merge(left=df_prep, right=df_dep, left_on='idAutor', right_on='id')
df_prep

,ano,dataApresentacao,descricaoTipo,ementa,ementaDetalhada,id_x,idTipo,idTipoAutor,justificativa,keywords,...,uriOrgaoNumerador,uriPropAnterior,uriPropPosterior,uriPropPrincipal,uriUltimoRelator,urlInteiroTeor,urnFinal,idAutor,id_y,nomeCivil
0,2011,2011-10-19T21:21,Projeto de Lei,"Altera a Lei nº 9.503, de 23 de setembro de 19...",NaN,524290,139,10000,NaN,"Alteração, Código de Trânsito Brasileiro, util...",...,NaN,NaN,NaN,https://dadosabertos.camara.leg.br/api/v2/prop...,NaN,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,160593,160593,PAULO WAGNER LEITE DANTAS
1,2011,2011-10-19T21:22,Projeto de Lei,Institui a meia-entrada para doadores de sangu...,NaN,524291,139,10000,NaN,"Concessão, doador, sangue, medula óssea, meia-...",...,NaN,NaN,NaN,https://dadosabertos.camara.leg.br/api/v2/prop...,NaN,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,160593,160593,PAULO WAGNER LEITE DANTAS
2,2011,2011-11-30T19:51,Projeto de Lei,Denomina Dona Militana o aeroporto construído ...,NaN,529389,139,10000,NaN,"Denominação, aeroporto, São Gonçalo do Amarant...",...,NaN,NaN,NaN,None,NaN,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,160593,160593,PAULO WAGNER LEITE DANTAS
3,2011,2011-11-30T19:52,Projeto de Lei,"Altera a Lei nº 9.503, de 23 de setembro de 19...",NaN,529393,139,10000,NaN,"Alteração, Código de Trânsito Brasileiro, fixa...",...,NaN,NaN,NaN,None,NaN,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,160593,160593,PAULO WAGNER LEITE DANTAS
4,2013,2013-03-20T17:50,Projeto de Lei,Obriga os mercados e supermercados a oferecere...,NaN,568467,139,10000,NaN,"Obrigatoriedade, mercado, supermercado, gratui...",...,NaN,NaN,NaN,https://dadosabertos.camara.leg.br/api/v2/prop...,NaN,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,160593,160593,PAULO WAGNER LEITE DANTAS
5,2013,2013-04-03T15:21,Projeto de Lei,Proíbe a venda de produtos derivados do tabaco...,NaN,570094,139,10000,NaN,"Proibição, venda, cigarro, produto, derivados,...",...,NaN,NaN,NaN,https://dadosabertos.camara.leg.br/api/v2/prop...,NaN,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,160593,160593,PAULO WAGNER LEITE DANTAS
6,2013,2013-09-18T15:12,Projeto de Lei,"Altera o Decreto-lei nº 2.848, de 7 de dezembr...",NaN,592362,139,10000,NaN,"Alteração, Código Penal, realização, castração...",...,NaN,NaN,NaN,https://dadosabertos.camara.leg.br/api/v2/prop...,NaN,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,160593,160593,PAULO WAGNER LEITE DANTAS
7,2013,2013-10-30T13:39,Proposta de Emenda à Constituição,Altera a redação do inciso III do art. 208 da ...,NaN,598619,136,10000,NaN,"Garantia, educação, inclusão, aluno, transtorn...",...,NaN,NaN,NaN,None,NaN,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,160593,160593,PAULO WAGNER LEITE DANTAS
8,2011,2011-04-06T20:25,Projeto de Lei,Dispõe sobre o Programa Escola Aberta nas esco...,NaN,497431,139,10000,NaN,"Implantação, Programa Escola Aberta, escola pú...",...,NaN,NaN,NaN,https://dadosabertos.camara.leg.br/api/v2/prop...,NaN,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,160593,160593,PAULO WAGNER LEITE DANTAS
9,2011,2011-05-04T20:20,Projeto de Lei,Dispõe sobre a reserva de vagas para portadore...,NaN,500744,139,10000,NaN,"Obrigatoriedade, instituição de ensino superio...",...,NaN,NaN,NaN,https://dadosabertos.camara.leg.br/api/v2/prop...,NaN,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,160593,160593,PAULO WAGNER LEITE DANTAS


Somando a quantidade de proposições por deputado

In [72]:
df_prep.nomeCivil.value_counts()

CARLOS FREDERICO THEODORO NADER                  647
CARLOS GOMES BEZERRA                             546
ANTONIO CARLOS DE MENDES THAME                   468
DARCI POMPEO DE MATTOS                           436
JOÃO ALBERTO FRAGA SILVA                         417
INOCÊNCIO GOMES DE OLIVEIRA                      393
ENIO EGON BERGMANN BACCI                         387
RÔMULO JOSÉ DE GOUVEIA                           306
CLEBER VERDE CORDEIRO MENDES                     286
ARNALDO FARIA DE SÁ                              285
CARLOS ALBERTO CAVALCANTE DE SOUZA               275
RUBENS BUENO                                     257
VALDIR COLATTO                                   251
LUIZ CARLOS JORGE HAULY                          244
HENRIQUE EDUARDO LYRA ALVES                      215
LINCOLN DINIZ PORTELA                            206
MARIA LAURA MONTEZA DE SOUZA CARNEIRO            201
CELSO UBIRAJARA RUSSOMANNO                       201
LUIZ GONZAGA PATRIOTA                         

Verificando a quantidade de proposições feita pelos depudados especificos


In [189]:
deputados = (df_prep.nomeCivil == 'JAIR MESSIAS BOLSONARO') | (df_prep.nomeCivil == 'JEAN WYLLYS DE MATOS SANTOS') | (df_prep.nomeCivil == 'MARCO ANTONIO Feliciano')
df_prep[deputados].nomeCivil.value_counts()

JAIR MESSIAS BOLSONARO         170
JEAN WYLLYS DE MATOS SANTOS     37
Name: nomeCivil, dtype: int64

Proposições por ano dos deputados

In [181]:
por_ano=df_prep[deputados].groupby(['nomeCivil','ano']).ano.count()
por_ano

nomeCivil                    ano 
JAIR MESSIAS BOLSONARO       1991     9
                             1992     7
                             1993     7
                             1994     5
                             1995     5
                             1996    15
                             1997    12
                             1998     4
                             2000     6
                             2001     2
                             2002     4
                             2003     2
                             2004     1
                             2005     7
                             2006    13
                             2007    10
                             2008     6
                             2009     7
                             2010     4
                             2011     1
                             2012     1
                             2013     8
                             2014    11
                             2015    10
      

Total de anos no cargo


In [187]:
por_ano.groupby(level='nomeCivil').count()

nomeCivil
JAIR MESSIAS BOLSONARO         26
JEAN WYLLYS DE MATOS SANTOS     7
Name: ano, dtype: int64

Vendo a média por ano


In [188]:
media_por_ano=por_ano.groupby(level='nomeCivil').sum() / por_ano.groupby(level='nomeCivil').count()
media_por_ano




nomeCivil
JAIR MESSIAS BOLSONARO         6.538462
JEAN WYLLYS DE MATOS SANTOS    5.285714
Name: ano, dtype: float64